# utils

In [1]:
import vectorbt as vbt

from vectorbt.utils import checks, config, decorators

In [2]:
import numpy as np
import pandas as pd
from numba import njit

In [3]:
v1 = 0
a1 = np.array([1])
a2 = np.array([1, 2, 3])
a3 = np.array([[1, 2, 3]])
a4 = np.array([[1], [2], [3]])
a5 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sr1 = pd.Series([1], index=pd.Index(['x1'], name='i1'))
print(sr1)
sr2 = pd.Series([1, 2, 3], index=pd.Index(['x2', 'y2', 'z2'], name='i2'))
print(sr2)
df1 = pd.DataFrame(
    [[1]], 
    index=pd.Index(['x3'], name='i3'), 
    columns=pd.Index(['a3'], name='c3'))
print(df1)
df2 = pd.DataFrame(
    [[1], [2], [3]], 
    index=pd.Index(['x4', 'y4', 'z4'], name='i4'), 
    columns=pd.Index(['a4'], name='c4'))
print(df2)
df3 = pd.DataFrame(
    [[1, 2, 3]], 
    index=pd.Index(['x5'], name='i5'), 
    columns=pd.Index(['a5', 'b5', 'c5'], name='c5'))
print(df3)
df4 = pd.DataFrame(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 
    index=pd.Index(['x6', 'y6', 'z6'], name='i6'), 
    columns=pd.Index(['a6', 'b6', 'c6'], name='c6'))
print(df4)

multi_i = pd.MultiIndex.from_arrays([['x7', 'y7', 'z7'], ['x8', 'y8', 'z8']], names=['i7', 'i8']) 
multi_c = pd.MultiIndex.from_arrays([['a7', 'b7', 'c7'], ['a8', 'b8', 'c8']], names=['c7', 'c8'])
df5 = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=multi_i, columns=multi_c)
print(df5)

i1
x1    1
dtype: int64
i2
x2    1
y2    2
z2    3
dtype: int64
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  4  5  6
z7 z8  7  8  9


## config

In [4]:
conf = config.Config({'a': 0, 'b': {'c': 1}})
print(conf)

conf['a'] = 2
print(conf)

try:
    conf['d'] = 2
    print(conf)
except:
    print('OK')
    
conf['b']['c'] = 2
print(conf)

try:
    conf['b']['d'] = 2
    print(conf)
except:
    print('OK')

{'a': 0, 'b': {'c': 1}}
{'a': 2, 'b': {'c': 1}}
OK
{'a': 2, 'b': {'c': 2}}
OK


In [5]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen=False)

conf['b']['d'] = 2
print(conf)

{'a': 0, 'b': {'c': 1, 'd': 2}}


In [6]:
print(config.merge_kwargs({'a': 1}, {'b': 2}))
print(config.merge_kwargs({'a': 1}, {'a': 2}))
print(config.merge_kwargs({'a': {'b': 2}}, {'a': {'c': 3}}))
print(config.merge_kwargs({'a': {'b': 2}}, {'a': {'b': 3}}))

{'a': 1, 'b': 2}
{'a': 2}
{'a': {'b': 2, 'c': 3}}
{'a': {'b': 3}}


## decorators

In [7]:
class G():
    @decorators.class_or_instancemethod
    def g(self_or_cls):
        if isinstance(self_or_cls, type):
            print("class")
        else:
            print("instance")
            
G.g()
G().g()

class
instance


In [8]:
class G():
    @decorators.cached_property(hello="world")
    def cache_me(self): return np.random.uniform(size=(10000, 10000))
    
G.cache_me.kwargs

{'hello': 'world'}

In [9]:
class G():
    @decorators.cached_property
    def cache_me(self): return np.random.uniform(size=(10000, 10000))
    
g = G()

In [10]:
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.75 s, sys: 309 ms, total: 2.06 s
Wall time: 2.21 s
CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 16.9 µs


In [11]:
dir(g)

['__cached_cache_me',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [12]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.71 s, sys: 460 ms, total: 2.17 s
Wall time: 2.61 s
CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.82 µs


In [13]:
G.cache_me.disabled = True
%time _ = g.cache_me
%time _ = g.cache_me
G.cache_me.disabled = False

CPU times: user 1.44 s, sys: 269 ms, total: 1.71 s
Wall time: 1.89 s
CPU times: user 2 s, sys: 504 ms, total: 2.5 s
Wall time: 3.26 s


In [14]:
vbt.defaults.caching = False
%time _ = g.cache_me
%time _ = g.cache_me
vbt.defaults.caching = True

CPU times: user 1.55 s, sys: 374 ms, total: 1.92 s
Wall time: 2.18 s
CPU times: user 1.23 s, sys: 287 ms, total: 1.52 s
Wall time: 1.52 s


In [15]:
class G():
    @decorators.cached_method(hello="world")
    def cache_me(self, a): return np.random.uniform(size=(10000, 10000)) * a

G.cache_me.kwargs

{'hello': 'world'}

In [16]:
class G():
    @decorators.cached_method
    def cache_me(self, a): return np.random.uniform(size=(10000, 10000)) * a

g = G()

In [17]:
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)

CPU times: user 1.31 s, sys: 270 ms, total: 1.58 s
Wall time: 1.59 s
CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 14.1 µs


In [18]:
dir(g)

['__cached_cache_me',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [19]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me(2)

CPU times: user 1.32 s, sys: 275 ms, total: 1.6 s
Wall time: 1.61 s


In [20]:
G.cache_me.disabled = True
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
G.cache_me.disabled = False

CPU times: user 1.34 s, sys: 213 ms, total: 1.55 s
Wall time: 1.56 s
CPU times: user 1.33 s, sys: 264 ms, total: 1.6 s
Wall time: 1.6 s


In [21]:
vbt.defaults.caching = False
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.defaults.caching = True

CPU times: user 1.36 s, sys: 267 ms, total: 1.62 s
Wall time: 1.64 s
CPU times: user 1.32 s, sys: 268 ms, total: 1.59 s
Wall time: 1.59 s


In [22]:
# Non-hashable arguments won't cache
%time _ = g.cache_me(np.asarray(2))
%time _ = g.cache_me(np.asarray(2))

CPU times: user 1.31 s, sys: 279 ms, total: 1.59 s
Wall time: 1.59 s
CPU times: user 1.3 s, sys: 271 ms, total: 1.58 s
Wall time: 1.58 s


In [23]:
class A:
    @decorators.custom_property(some_key=0)
    def a(self): pass

class B:
    @decorators.cached_property(some_key=0, child_cls=A)
    def a(self): pass
    
    @decorators.custom_method(some_key=1)
    def b(self): pass
    
class C:
    @decorators.cached_method(some_key=0, child_cls=B)
    def b(self): pass
    
    @decorators.custom_property(some_key=1)
    def c(self): pass

In [24]:
str(decorators.traverse_attr_kwargs(C))

"{'b': {'some_key': 0, 'child_cls': <class '__main__.B'>, 'child_attrs': {'a': {'some_key': 0, 'child_cls': <class '__main__.A'>, 'child_attrs': {'a': {'some_key': 0}}}, 'b': {'some_key': 1}}}, 'c': {'some_key': 1}}"

In [25]:
decorators.traverse_attr_kwargs(C, key='some_key')

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {'a': {'some_key': 0}}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

In [26]:
decorators.traverse_attr_kwargs(C, key='some_key', value=1)

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

In [27]:
decorators.traverse_attr_kwargs(C, key='some_key', value=(0, 1))

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {'a': {'some_key': 0}}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

## checks

In [28]:
print(checks.is_series(v1))
print(checks.is_series(a1))
print(checks.is_series(sr1))
print(checks.is_series(df1))

False
False
True
False


In [29]:
print(checks.is_frame(v1))
print(checks.is_frame(a1))
print(checks.is_frame(sr1))
print(checks.is_frame(df1))

False
False
False
True


In [30]:
print(checks.is_pandas(v1))
print(checks.is_pandas(a1))
print(checks.is_pandas(sr1))
print(checks.is_pandas(df1))

False
False
True
True


In [31]:
print(checks.is_array(v1))
print(checks.is_array(a1))
print(checks.is_array(sr1))
print(checks.is_array(df1))

False
True
True
True


In [32]:
print(checks.is_numba_func(lambda x: x))
print(checks.is_numba_func(njit(lambda x: x)))

False
True


In [33]:
print(checks.is_hashable(2))
print(checks.is_hashable(np.asarray(2)))

True
False


In [34]:
checks.assert_value_in(0, (0, 1))

In [35]:
checks.assert_numba_func(njit(lambda x: x))

In [36]:
checks.assert_not_none(v1)

In [37]:
checks.assert_type(v1, int)
checks.assert_type(a1, np.ndarray)
checks.assert_type(sr1, (np.ndarray, pd.Series))

In [38]:
checks.assert_same_type(v1, v1)
checks.assert_same_type(a1, a2)
checks.assert_same_type(sr1, sr1)
checks.assert_same_type(df1, df2)

In [39]:
checks.assert_dtype(a1, np.int)

In [40]:
checks.assert_same_dtype(v1, a1)
checks.assert_same_dtype(a1, df1)
checks.assert_same_dtype(df1, df2)
checks.assert_same_dtype(df2, df3)

In [41]:
checks.assert_ndim(v1, 0)
checks.assert_ndim(a1, 1)
checks.assert_ndim(df1, 2)

In [42]:
checks.assert_same_len([[1]], [[2]])

In [43]:
checks.assert_same_shape(a1, sr1)
checks.assert_same_shape(df2, df4, axis=0)
checks.assert_same_shape(df3, df4, axis=1)
checks.assert_same_shape(df2, df3, axis=(0, 1))

In [44]:
checks.assert_same_index(df3, df3)

In [45]:
checks.assert_same_columns(df3, df3)

In [46]:
checks.assert_same_meta(df3, df3)

In [47]:
checks.assert_same(df3, df3)

In [48]:
checks.assert_level_not_exists(df3.columns, 'a')